In [0]:
!pip install rarfile
!pip install transformers

     |████████████████████████████████| 122kB 3.0MB/s 
  Created wheel for rarfile: filename=rarfile-3.1-cp36-none-any.whl size=24908 sha256=3c7f8d37b1ca87e347e29ec710ddc08d4d7927cb96031e1c98927387c2900157
  Stored in directory: /root/.cache/pip/wheels/23/3c/c8/0215b6a5079492eff3be3f545ae0b0c4a66734c35c9e444eac
Successfully built rarfile
     |████████████████████████████████| 481kB 2.9MB/s 
     |████████████████████████████████| 3.1MB 78.1MB/s 
     |████████████████████████████████| 870kB 62.2MB/s 
     |████████████████████████████████| 1.0MB 44.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=b9c2300f3c880aa3916578400f707de9d5fab5bb54d2df9fef73af13121a4388
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
   
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '1Vd1eT_YVwdijl1GZqQdP1Ep0OGS28tay'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('longdata.rar')
file_id = '13vKE-OHTfeYzZVGxUOjjPQ_lzPljTeH0'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('sumdata.rar')
file_id = '16zame2wNb25A4BP67VxON5ft-mJ3N_2n'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('trains.csv')
file_id = '1Gd20wkDA_vpxdbHGV45RoVQZkmyynamu'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train_neu.csv')


In [0]:
file_id = '15z3JquZZXwWF0X4Vue3sh6Rvbup1lz9k'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('trains.csv')


In [0]:

import rarfile

rf = rarfile.RarFile('longdata.rar')        
rf.extractall()               
rf.close()
rf = rarfile.RarFile('sumdata.rar')        
rf.extractall()               
rf.close()

In [0]:
import numpy as np
import pandas as pd
import os
from operator import itemgetter
import re

path1 = os.listdir('./longdata')
path2 = os.listdir('./sumdata')
path2.remove( '.ipynb_checkpoints')
path1.sort()
path2 = sorted(path2, key = lambda x : int(x.replace('_','').split('.')[0]))


def readdata(data, stopword, long):
    tt = ''
    if long == 'long':
        with open('./longdata/' + data,'r', encoding='big5') as t921:
            for l in t921:
                if l != '' :
                    tt += l
    elif long == 'short':    
        with open('./sumdata/' + data,'r') as t921:
            for l in t921:
                if l != '' :
                    tt += l            
    for c in stopword2:
        tt = tt.replace(c,'')
    tt = tt.replace('消費者物價指數','CPI')
    tt = tt.replace('CPICPI','CPI')
    tt = tt.replace('製造業採購經理人指數','PMI')
    tt = tt.replace('製造業經理人指數','NMI')
    #tt = tt.split('，')
    tt = re.split(r'[；。]', tt)
    return tt            

stopword = ['\n','\r','\t','\u3000','：','；',' ','、','）','(','（',')','.','亦','在','將','至',
            '的','但','卻','及','為','且','呈','以','則','與','若','仍','惟','之','」','「','<','-',
           'http//wwwcbcgovtw>98']
stopword2 = ['\n','\r','\t','\u3000']           
documents1 = []
documents2 = []

for i in path1:
    documents1 += readdata(i, stopword2, long = 'long')
for i in path2:
    documents2 += readdata(i, stopword2, long = 'short')
    
print(f'會議記錄 : {len(documents1)}句')
print(f'新聞稿 : {len(documents2)}句')    

會議記錄 : 1260句
新聞稿 : 2990句


In [0]:
trains = pd.read_csv('trains.csv', encoding = 'utf-8')
print(trains.shape)
trains.head()

(4250, 3)


,sentent,non_sentent,labels
0,"['討論', '事項', '：', '國內外', '經濟', '金融', '情勢', '與'...",討論事項：國內外經濟金融情勢與本行貨幣政策提請討論,neutral
1,"['壹', '、', '國內外', '經濟', '金融', '情勢', '1', '經濟',...",壹、國內外經濟金融情勢1經濟研究處說明：一、 國際經濟金融情勢隨全球製造業及貿易活動漸趨活絡...,positive
2,"['多數', '經濟體', '展望', '趨於', '樂觀']",多數經濟體展望趨於樂觀,positive
3,"['其中', '美國', '、', '日本', '、', '歐元區', '等', '經濟',...",其中美國、日本、歐元區等經濟皆穩健成長亞洲經濟體成長率多高於上年惟中國大陸成長則略緩,neutral
4,"['先進', '經濟體', '因', '經濟', '復甦', '預期', '本年', '通膨...",先進經濟體因經濟復甦預期本年通膨溫和成長,neutral


In [0]:
trains['labels'].value_counts()

neutral     2373
negative    1023
positive     531
unk          323
Name: labels, dtype: int64

In [0]:
label_dict = {'positive':0, 'negative':1, 'neutral':2, 'unk':2}
trains['labels'] = trains['labels'].map(label_dict)
trains.head()

,sentent,non_sentent,labels
0,"['討論', '事項', '：', '國內外', '經濟', '金融', '情勢', '與'...",討論事項：國內外經濟金融情勢與本行貨幣政策提請討論,2
1,"['壹', '、', '國內外', '經濟', '金融', '情勢', '1', '經濟',...",壹、國內外經濟金融情勢1經濟研究處說明：一、 國際經濟金融情勢隨全球製造業及貿易活動漸趨活絡...,0
2,"['多數', '經濟體', '展望', '趨於', '樂觀']",多數經濟體展望趨於樂觀,0
3,"['其中', '美國', '、', '日本', '、', '歐元區', '等', '經濟',...",其中美國、日本、歐元區等經濟皆穩健成長亞洲經濟體成長率多高於上年惟中國大陸成長則略緩,2
4,"['先進', '經濟體', '因', '經濟', '復甦', '預期', '本年', '通膨...",先進經濟體因經濟復甦預期本年通膨溫和成長,2


In [0]:
train2 = pd.DataFrame()
train2['sentent'] = documents1 + documents2
train2['labels'] = trains['labels'].map(lambda x : int(x)).values
train2['labels'].value_counts()

2    2696
1    1023
0     531
Name: labels, dtype: int64

In [0]:
train_neu = pd.read_csv('train_neu.csv', encoding = 'big5', header = None)
train_neu.fillna(6)
trains_neu = pd.DataFrame()
trains_neu['sentent'] = train_neu[train_neu[0] == 2.0][1]
trains_neu['labels'] = train_neu[train_neu[0] == 2.0][0].map(lambda x : int(x))
trains_neu.head()

,sentent,labels
0,討論事項：國內外經濟金融情勢與本行貨幣政策提請討論,2
4,貿易保護主義抬頭可能影響全球貿易復甦,2
6,國內外主要機構普遍預測本年經濟成長率介於 2.04%至 2.20%,2
11,如同多數國家實質利率呈現負值,2
14,銀行授信方面5 月放款與投資成長趨緩年增 5.06%主要係因應所得稅款解繳銀行減持投資部位所致,2


In [0]:
train_set = train2[train2['labels'] != 2]
val_set = train_set.iloc[-250:]
train_set = train_set.iloc[:-250]
train_set = pd.concat([train_set, trains_neu.iloc[:100]])
val_set = pd.concat([val_set, trains_neu.iloc[100:]])
index_neu = trains_neu['sentent'].values
test_set = train2[train2['labels'] == 2]

print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

(1404, 2)
(327, 2)
(2696, 2)


In [0]:
PRETRAINED_MODEL_NAME = 'bert-base-chinese'

tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print(f'PyTorch版本 : {torch.__version__}')

PyTorch版本 : 1.4.0


In [0]:
vocab = tokenizer.vocab
print(f'字典大小 : {len(vocab)}')

字典大小 : 21128


In [0]:
from torch.utils.data import Dataset

class SentimentForCB(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
     
        
    def __getitem__ (self, idx):
        cont = '[CLS]' + self.data['sentent'].iloc[idx]
        label = self.data['labels'].iloc[idx]
        cont = self.tokenizer.tokenize(cont)
        cont = self.tokenizer.convert_tokens_to_ids(cont)
        cont = torch.tensor(cont)
        label = torch.tensor(label)
        
        return (cont, label)
    
    def __len__(self):
        return len(self.data)

In [0]:
from torch.nn.utils.rnn import pad_sequence

def create_minibatch(samples):
    
    tokens = [s[0] for s in samples]
    tokens = pad_sequence(tokens, batch_first=True)
    
    labels = torch.stack([l[1] for l in samples])
    masked = torch.zeros(tokens.shape)
    masked = masked.masked_fill(tokens != 0, 1)
    
    return tokens, masked, labels

In [0]:
def get_prediction(model, dataloader):
    correct = 0
    total = 0
    prediction = None
    use_gpu = torch.cuda.is_available()
    with torch.no_grad():
        for tok, mas, lb in dataloader:
            if use_gpu:
                tok = tok.cuda()
                mas = mas.cuda()
                lb = lb.cuda()                
            outputs = model(input_ids = tok, attention_mask = mas)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
        
            correct += (pred == lb).sum().item()
            total += lb.shape[0]
        
            if prediction is None:
                prediction = pred
            else:
                prediction = torch.cat((prediction, pred))
        acc = correct/total
    
    return (prediction, acc)

In [0]:
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import time

use_gpu = torch.cuda.is_available()

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME, num_labels = 3)
clear_output()

train_dataset = SentimentForCB(train_set, tokenizer)
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True, collate_fn = create_minibatch)
val_dataset = SentimentForCB(val_set, tokenizer)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = False, collate_fn=  create_minibatch)

if use_gpu:
    model.cuda()

start = time.time()

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = []
    correct = 0
    total = 0
    for i, (tok, mas, lb) in enumerate(train_loader):
        if use_gpu:
            tok = tok.cuda()
            mas = mas.cuda()
            lb = lb.cuda()
        batch_size = tok.shape[0]    
        optimizer.zero_grad()
        outputs = model(input_ids = tok, attention_mask = mas)
        #logits = outputs[0]
        _, pred = torch.max(outputs[0], 1)
        loss = loss_fn(outputs[0],lb)
        loss.backward()
        optimizer.step()
        correct += (pred == lb).sum().item()
        total += lb.shape[0]
        train_loss.append(loss.item())
    acc = correct/total 
    
    print(f'train [epoch{epoch+1:2d}], acc = {acc:.3f}, loss = {np.mean(train_loss)}')

    model.eval()
    with torch.no_grad():
        train_loss = []
        correct = 0
        total = 0
        for i, (tok, mas, lb) in enumerate(val_loader):
            if use_gpu:
                tok = tok.cuda()
                mas = mas.cuda()
                lb = lb.cuda()   
            batch_size = tok.shape[0]        
            outputs = model(input_ids = tok, attention_mask = mas)
           # logits = outputs[0]     
            _, pred = torch.max(outputs[0], 1)
            loss = loss_fn(outputs[0],lb)
            train_loss.append(loss.item())
            correct += (pred == lb).sum().item()
            total += lb.shape[0]
        acc = correct/total    
        print(f'valid [epoch{epoch+1:2d}], acc = {acc:.3f}, loss = {np.mean(train_loss)}')   
    if acc > 0.9:
      model_path = f'model_{epoch+1}.pkl'
      torch.save(model.state_dict(), model_path)
      print(f'model_{epoch+1} saved')
    print(f'spend {time.time()-start:3.1f}s')

train [epoch 1], acc = 0.667, loss = 0.730697882107713
valid [epoch 1], acc = 0.667, loss = 0.5795794931196031
spend 33.7s
train [epoch 2], acc = 0.865, loss = 0.36322202398018405
valid [epoch 2], acc = 0.865, loss = 0.36671888047740575
spend 68.1s
train [epoch 3], acc = 0.952, loss = 0.1713488853367215
valid [epoch 3], acc = 0.869, loss = 0.4142872921767689
spend 101.7s
train [epoch 4], acc = 0.972, loss = 0.09305391426790845
valid [epoch 4], acc = 0.930, loss = 0.28428964884508223
model_4 saved
spend 137.0s
train [epoch 5], acc = 0.984, loss = 0.06157800618728453
valid [epoch 5], acc = 0.920, loss = 0.32098570253167835
model_5 saved
spend 172.0s
train [epoch 6], acc = 0.993, loss = 0.0285747169083069
valid [epoch 6], acc = 0.887, loss = 0.4756282912123771
spend 205.9s
train [epoch 7], acc = 0.995, loss = 0.02446222641050223
valid [epoch 7], acc = 0.920, loss = 0.3598236647390184
model_7 saved
spend 241.2s
train [epoch 8], acc = 0.995, loss = 0.015285781330683014
valid [epoch 8], acc 

In [0]:

test_dataset = SentimentForCB(test_set, tokenizer)
test_loader = DataLoader(test_dataset, batch_size = 16, collate_fn=create_minibatch)

model.load_state_dict(torch.load('model_4.pkl'))
model.cuda()
with torch.no_grad():
    predict_value = []
    for tok, mas, lb in test_loader:
        if use_gpu:
            tok = tok.cuda()
            mas = mas.cuda()
            lb = lb.cuda()
        outputs = model(input_ids = tok, attention_mask = mas)
        
        _, pred = torch.max(outputs[0], 1)
        predict_value += pred.tolist()

In [0]:
predict_value[:10]

[1, 0, 0, 0, 1, 1, 1, 0, 2, 0]

In [0]:
test_set['labels'] = predict_value
test_set.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sentent,labels
0,討論事項：國內外經濟金融情勢與本行貨幣政策，提請討論,1
3,其中，美國、日本、歐元區等經濟皆穩健成長，亞洲經濟體成長率多高於上年，惟中國大陸成長則略緩,0
4,先進經濟體因經濟復甦，預期本年通膨溫和成長,0
5,新興市場經濟體則因部分國家幣值回升及政策調控，高通膨現象可望緩解,0
7,惟全球政策不確定性(EPU 指數)上升，且 Fed 期望自本年下半年開始縮減資產負債表規模，...,1


In [0]:
test_set['labels'].value_counts()

0    1326
1    1109
2     261
Name: labels, dtype: int64

In [0]:
test_set.to_csv('test_set.csv', index = None)
files.download('test_set.csv')